In [2]:
import geopandas as gpd

gdf = gpd.read_parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\NLSDB\2025_03-03_NLSDB.gdb\CaseLands_03032025_1330.parquet")
gdf.head()

,OBJECTID,CSE_NR,LEG_CSE_NR,REC_TYPE_CSE_GRP,BLM_PROD,CSE_TYPE_NR,CSE_LND_STATUS,CSE_LND_STATUS_DT,CSE_LND_NR,US_RIGHTS,...,SF_ID,SF_CL_ID,ID,AGG_CLS_ID,QLTY_CK,Created,Modified,Shape_Length,Shape_Area,Shape
0,1,AKAK106538478,AKFF 015208,Land Transfer - Grants,GENERAL GRANT,262730,Patented,1985-08-29 23:59:00+00:00,LLD-32629751,DITCHES OR CANALS,...,a02SJ000002HUveYAG,a08SJ000007xf4kYAA,0,201969,None,2024-10-06 04:55:15+00:00,2024-10-06 04:55:15+00:00,0.097572,0.000496,"MULTIPOLYGON (((-153.83207 65.19195, -153.8664..."
1,2,AKAK106538478,AKFF 015208,Land Transfer - Grants,GENERAL GRANT,262730,Patented,1985-08-29 23:59:00+00:00,LLD-32629752,DITCHES OR CANALS,...,a02SJ000002HUveYAG,a08SJ000007xf4lYAA,1,201969,None,2024-10-06 04:55:15+00:00,2024-10-06 04:55:15+00:00,0.097533,0.000496,"MULTIPOLYGON (((-154.44821 65.17750, -154.4825..."
2,3,AKAK106538478,AKFF 015208,Land Transfer - Grants,GENERAL GRANT,262730,Patented,1985-08-29 23:59:00+00:00,LLD-32629761,DITCHES OR CANALS,...,a02SJ000002HUveYAG,a08SJ000007xf4uYAA,10,201969,None,2024-10-06 04:55:15+00:00,2024-10-06 04:55:15+00:00,0.097496,0.000495,"MULTIPOLYGON (((-154.51685 65.16307, -154.5511..."
3,4,AKAK106632503,AKFF 043729,Land Transfer - Grants,GENERAL GRANT,262730,Tentatively Approved,1984-11-07 23:59:00+00:00,LLD-32800552,None,...,a02SJ000002HjrXYAS,a08SJ000007xraZYAQ,100,162062,None,2024-10-06 04:55:16+00:00,2024-10-06 04:55:16+00:00,0.050842,0.000159,"MULTIPOLYGON (((-144.49428 64.06979, -144.5052..."
4,5,AKAK106670513,AKF 019711 B,Lease - Oil and Gas,O&G NONCOMPETITIVE PUBLIC DOMAIN LEASE PRE 1987,311111,Terminated,1963-04-01 23:59:01.001000+00:00,LLD-32251984,None,...,a02SJ000002H0tgYAC,a08SJ000007yIrSYAU,1000,104898,None,2024-10-06 04:55:16+00:00,2024-10-06 04:55:16+00:00,0.088670,0.000432,"MULTIPOLYGON (((-162.75816 61.16609, -162.7843..."


In [ ]:
gdf = gpd.read_file(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\NLSDB\2025_03-03_NLSDB.gdb\nlsdb_03032025.gdb", layer = 'CaseLands_03032025_1330')
gdf.head()